In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-
import time

import os
import numpy as np
# import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, GRU,Activation
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
import pandas as pd

In [7]:
# get texts data
category2idx = {'Japan_Travel': 0, 'KR_ENTERTAIN': 1, 'Makeup': 2, 'Tech_Job':  3, 'WomenTalk': 4,
                  'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}

train_df_sample = pd.read_pickle('train.pkl').sample(frac=1, random_state=123)
# train_df_sample = pd.concat([train_pickle_df.text]).sample(frac=1)

In [8]:
train_texts = train_df_sample.values
label_list = to_categorical(train_df_sample.category)

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values

In [9]:
train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])
# print(len(train_texts_list), train_texts_list[0:10])
train_texts = train_texts_list

In [10]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec1.model")
# print(type(answer))
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)
# print(wvv_keys_list[:10]) #['櫻花林', '好比', '考科', '床上', '一點現', '記住', '寶寶的', '柔嫩', '不規則', '朴智妍']

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
# max_doc_word_length = max(len(l) for l in train_texts)
max_doc_word_length = 250
sequences1 = pad_sequences(sequences, maxlen=max_doc_word_length, padding='post')
word_index = tokenizer.word_index
print("Found %s unique tokens" % len(word_index))
# data = pad_sequences(sequences)
# print("Shape of data tensor:" , data.shape)

Found 232236 unique tokens


In [12]:
vocab_size = len(word_index) + 1
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 250))
for word, i in word_index.items():
    if word in wvv_keys_list:
        embedding_vector = answer[word]
#         if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.zeros((1, 250))

In [13]:
del answer

In [14]:
embedding_layer = Embedding(input_dim= embedding_matrix.shape[0],
                            output_dim= embedding_matrix.shape[1],
                            weights=[embedding_matrix], 
                            trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(GRU(16))    
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 250)         58059250  
_________________________________________________________________
gru_1 (GRU)                  (None, 16)                12816     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1700      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 58,084,876
Trainable params: 25,626
Non-trainable params: 58,059,250
_________________________________________________________________


In [15]:
history = model.fit(x = sequences1, y = label_list, 
                    validation_split=0.1, 
                    batch_size= 1620,
                    epochs = 100, verbose = 1)

Train on 8100 samples, validate on 900 samples
Epoch 1/200
8100/8100 [==============================] - 13s 2ms/step - loss: 2.2926 - acc: 0.1159 - val_loss: 2.2811 - val_acc: 0.1556
Epoch 2/200
8100/8100 [==============================] - 13s 2ms/step - loss: 2.2636 - acc: 0.1474 - val_loss: 2.2576 - val_acc: 0.1533
Epoch 3/200
8100/8100 [==============================] - 13s 2ms/step - loss: 2.2336 - acc: 0.1614 - val_loss: 2.2422 - val_acc: 0.1511
Epoch 4/200
8100/8100 [==============================] - 13s 2ms/step - loss: 2.2149 - acc: 0.1653 - val_loss: 2.2298 - val_acc: 0.1511
Epoch 5/200
8100/8100 [==============================] - 13s 2ms/step - loss: 2.1950 - acc: 0.1705 - val_loss: 2.2106 - val_acc: 0.1622
Epoch 6/200
8100/8100 [==============================] - 13s 2ms/step - loss: 2.1747 - acc: 0.1852 - val_loss: 2.1933 - val_acc: 0.1644
Epoch 7/200
8100/8100 [==============================] - 13s 2ms/step - loss: 2.1540 - acc: 0.1926 - val_loss: 2.1743 - val_acc: 0.1700
E

8100/8100 [==============================] - 13s 2ms/step - loss: 0.8809 - acc: 0.6347 - val_loss: 1.0794 - val_acc: 0.5978
Epoch 61/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.8806 - acc: 0.6472 - val_loss: 1.1170 - val_acc: 0.6011
Epoch 62/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.8824 - acc: 0.6405 - val_loss: 1.0705 - val_acc: 0.5967
Epoch 63/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.8560 - acc: 0.6451 - val_loss: 1.0632 - val_acc: 0.6044
Epoch 64/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.8428 - acc: 0.6642 - val_loss: 1.0611 - val_acc: 0.6067
Epoch 65/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.8385 - acc: 0.6502 - val_loss: 1.0526 - val_acc: 0.6122
Epoch 66/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.8333 - acc: 0.6591 - val_loss: 1.0517 - val_acc: 0.6144
Epoch 67/200
8100/8100 [==============================

Epoch 120/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.5711 - acc: 0.7743 - val_loss: 1.0419 - val_acc: 0.6889
Epoch 121/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.5866 - acc: 0.7657 - val_loss: 1.0421 - val_acc: 0.6911
Epoch 122/200
8100/8100 [==============================] - 15s 2ms/step - loss: 0.5739 - acc: 0.7764 - val_loss: 1.0593 - val_acc: 0.6878
Epoch 123/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.5586 - acc: 0.7885 - val_loss: 1.0234 - val_acc: 0.6944
Epoch 124/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.5645 - acc: 0.7826 - val_loss: 1.0484 - val_acc: 0.7011
Epoch 125/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.5599 - acc: 0.7856 - val_loss: 1.0198 - val_acc: 0.6878
Epoch 126/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.5427 - acc: 0.7959 - val_loss: 1.0283 - val_acc: 0.6922
Epoch 127/200
8100/8100 [=========

8100/8100 [==============================] - 15s 2ms/step - loss: 0.3976 - acc: 0.8794 - val_loss: 1.0057 - val_acc: 0.7522
Epoch 180/200
8100/8100 [==============================] - 14s 2ms/step - loss: 0.3860 - acc: 0.8904 - val_loss: 1.0171 - val_acc: 0.7356
Epoch 181/200
8100/8100 [==============================] - 14s 2ms/step - loss: 0.3917 - acc: 0.8841 - val_loss: 1.0186 - val_acc: 0.7467
Epoch 182/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.3865 - acc: 0.8864 - val_loss: 1.0244 - val_acc: 0.7456
Epoch 183/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.3806 - acc: 0.8912 - val_loss: 1.0336 - val_acc: 0.7356
Epoch 184/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.3870 - acc: 0.8838 - val_loss: 1.0229 - val_acc: 0.7489
Epoch 185/200
8100/8100 [==============================] - 13s 2ms/step - loss: 0.3766 - acc: 0.8930 - val_loss: 1.0278 - val_acc: 0.7422
Epoch 186/200
8100/8100 [=======================

In [16]:
# loss_func_name = 'categorical_crossentropy'
# for value in history.history["categorical_crossentropy"]:
#     print(history)
model.save('my_model.h5') 

In [17]:
# evaluate the model
loss_accuracy = model.evaluate(sequences1[0:100], label_list[0:100], verbose=1)
print(type(loss_accuracy), loss_accuracy)

100/100 [==============================] - 0s 2ms/step
<class 'list'> [0.5238227415084838, 0.89]


In [18]:
test_tokenizer = Tokenizer()
test_tokenizer.fit_on_texts(test_texts)
test_sequences = test_tokenizer.texts_to_sequences(test_texts)
test_sequences1 = pad_sequences(test_sequences, maxlen=max_doc_word_length, padding='post')

In [19]:
predict_res = model.predict(test_sequences1, batch_size= 32, verbose=0)
print(len(predict_res), predict_res)

final_res = []
for pre_res in predict_res:
    final_res.append(np.argmax(pre_res))
print(final_res)

1000 [[1.9320464e-07 8.7407464e-01 4.7745794e-05 ... 9.1585671e-06
  7.9019396e-03 4.9097063e-05]
 [1.9736557e-07 8.7131131e-01 4.7050176e-05 ... 9.5462510e-06
  8.1725912e-03 5.0641796e-05]
 [8.4549612e-07 4.0742087e-01 8.3269961e-06 ... 2.3526109e-04
  1.0035634e-01 4.8514875e-04]
 ...
 [1.7587967e-02 9.4447612e-05 1.6718761e-16 ... 1.2588315e-04
  5.4116012e-03 9.7629148e-01]
 [5.3810783e-07 1.7043152e-04 2.0412126e-10 ... 2.2476161e-02
  8.3434522e-01 2.7035619e-03]
 [1.2116678e-08 6.6484714e-11 2.1173148e-05 ... 7.7167111e-20
  1.0814788e-08 9.7831416e-16]]
[1, 1, 4, 4, 4, 1, 7, 8, 2, 7, 4, 1, 1, 8, 8, 4, 2, 8, 1, 7, 8, 1, 9, 1, 1, 8, 8, 1, 4, 1, 1, 6, 7, 4, 1, 2, 4, 5, 1, 8, 1, 4, 1, 4, 1, 2, 4, 1, 1, 6, 1, 2, 4, 8, 8, 6, 2, 1, 1, 6, 4, 4, 1, 1, 1, 4, 4, 5, 7, 2, 1, 2, 1, 1, 1, 2, 4, 2, 1, 1, 6, 7, 0, 1, 5, 8, 6, 8, 2, 1, 8, 4, 5, 1, 2, 1, 4, 1, 4, 2, 7, 5, 9, 8, 8, 1, 7, 1, 7, 5, 9, 4, 1, 6, 8, 4, 4, 8, 7, 7, 8, 6, 4, 8, 8, 2, 1, 1, 8, 4, 1, 7, 1, 8, 8, 8, 2, 7, 4, 2, 1, 4, 4, 0

In [20]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "result001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,category" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(value) + '\n')
        ids += 1

1000


In [21]:
# k=0
# print(train_texts[2010:2020])

In [22]:
# print(len(test_sequences1))
# print(len(test_sequences1[0]))
# print(len(label_list))
# print(len(label_list[0]))
# old_label = label_list[0]
# print(type(label_list[0]))
# print(type(old_label))
# for label1 in label_list:
#     if not np.array_equal(label1 , old_label):
#         print(label1)
#         old_label = label1

In [23]:
# print(label_list[2010:2020])